In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install torch-geometric


Looking in indexes: https://download.pytorch.org/whl/cu121


In [2]:
# kan_layer.py
import torch
import torch.nn as nn
import numpy as np

class NaiveFourierKANLayer(nn.Module):
    def __init__(self, inputdim, outdim, gridsize=300, addbias=True):
        super(NaiveFourierKANLayer, self).__init__()
        self.gridsize = gridsize
        self.addbias = addbias
        self.inputdim = inputdim
        self.outdim = outdim

        self.fouriercoeffs = nn.Parameter(torch.randn(2, outdim, inputdim, gridsize) /
                                          (np.sqrt(inputdim) * np.sqrt(self.gridsize)))
        if self.addbias:
            self.bias = nn.Parameter(torch.zeros(1, outdim))

    def forward(self, x):
        xshp = x.shape
        outshape = xshp[0:-1] + (self.outdim,)
        x = x.view(-1, self.inputdim)
        k = torch.reshape(torch.arange(1, self.gridsize + 1, device=x.device), (1, 1, 1, self.gridsize))
        xrshp = x.view(x.shape[0], 1, x.shape[1], 1)
        c = torch.cos(k * xrshp)
        s = torch.sin(k * xrshp)
        c = torch.reshape(c, (1, x.shape[0], x.shape[1], self.gridsize))
        s = torch.reshape(s, (1, x.shape[0], x.shape[1], self.gridsize))
        y = torch.einsum("dbik,djik->bj", torch.concat([c, s], axis=0), self.fouriercoeffs)
        if self.addbias:
            y += self.bias
        y = y.view(outshape)
        return y


In [3]:
# utils.py
import torch
import numpy as np

def index_to_mask(index, size):
    mask = torch.zeros(size, dtype=torch.bool, device=index.device)
    mask[index] = 1
    return mask

def random_disassortative_splits(labels, num_classes, trn_percent=0.6, val_percent=0.2):
    labels, num_classes = labels.cpu(), num_classes.cpu().numpy()
    indices = []
    for i in range(num_classes):
        index = torch.nonzero((labels == i)).view(-1)
        index = index[torch.randperm(index.size(0))]
        indices.append(index)

    percls_trn = int(round(trn_percent * (labels.size()[0] / num_classes)))
    val_lb = int(round(val_percent * labels.size()[0]))
    train_index = torch.cat([i[:percls_trn] for i in indices], dim=0)

    rest_index = torch.cat([i[percls_trn:] for i in indices], dim=0)
    rest_index = rest_index[torch.randperm(rest_index.size(0))]

    train_mask = index_to_mask(train_index, size=labels.size()[0])
    val_mask = index_to_mask(rest_index[:val_lb], size=labels.size()[0])
    test_mask = index_to_mask(rest_index[val_lb:], size=labels.size()[0])

    return train_mask, val_mask, test_mask


In [8]:
import random
from torch_geometric.datasets import Planetoid, WebKB
import torch_geometric.transforms as T
from torch_geometric.utils import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class KanGNN(torch.nn.Module):
    def __init__(self, in_feat, hidden_feat, out_feat, grid_feat, num_layers, use_bias=False):
        super().__init__()
        self.num_layers = num_layers
        self.lin_in = nn.Linear(in_feat, hidden_feat, bias=use_bias)
        self.lins = torch.nn.ModuleList()
        for i in range(num_layers):
            self.lins.append(NaiveFourierKANLayer(hidden_feat, hidden_feat, grid_feat, addbias=use_bias))
        self.lins.append(nn.Linear(hidden_feat, out_feat, bias=False))

    def forward(self, x, adj):
        x = self.lin_in(x)
        for layer in self.lins[:self.num_layers - 1]:
            x = layer(spmm(adj, x))
        x = self.lins[-1](x)
        return x.log_softmax(dim=-1)

def train(args, feat, adj, label, mask, model, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(feat, adj)
    pred, true = out[mask], label[mask]
    loss = F.nll_loss(pred, true)
    acc = int((pred.argmax(dim=-1) == true).sum()) / int(mask.sum())
    loss.backward()
    optimizer.step()
    return acc, loss.item()

@torch.no_grad()
def eval(args, feat, adj, model):
    model.eval()
    with torch.no_grad():
        pred = model(feat, adj)
    pred = pred.argmax(dim=-1)
    return pred

class Args:
    path = './data/'
    name = 'Cora'
    logger_path = 'logger/esm'
    dropout = 0.0
    hidden_size = 256
    grid_size = 200
    n_layers = 2
    epochs = 1000
    early_stopping = 100
    seed = 42
    lr = 5e-4

args = Args()

args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

transform = T.Compose([T.NormalizeFeatures(), T.GCNNorm(), T.ToSparseTensor()])

print(f'run experiments on {args.name} dataset')

if args.name in {'Cora', 'Pubmed'}:
    dataset = Planetoid(args.path, args.name, transform=transform)[0]
elif args.name in {'Cornell'}:
    dataset = WebKB(args.path, args.name, transform=transform)[0]

in_feat = dataset.num_features
out_feat = max(dataset.y) + 1

model = KanGNN(
    in_feat=in_feat,
    hidden_feat=args.hidden_size,
    out_feat=out_feat,
    grid_feat=args.grid_size,
    num_layers=args.n_layers,
    use_bias=False,
).to(args.device)

optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

adj = dataset.adj_t.to(args.device)
feat = dataset.x.float().to(args.device)
label = dataset.y.to(args.device)

trn_mask, val_mask, tst_mask = random_disassortative_splits(label, out_feat)
trn_mask, val_mask, tst_mask = trn_mask.to(args.device), val_mask.to(args.device), tst_mask.to(args.device)

for epoch in range(args.epochs):
    trn_acc, trn_loss = train(args, feat, adj, label, trn_mask, model, optimizer)
    pred = eval(args, feat, adj, model)
    val_acc = int((pred[val_mask] == label[val_mask]).sum()) / int(val_mask.sum())
    tst_acc = int((pred[tst_mask] == label[tst_mask]).sum()) / int(tst_mask.sum())

    print(f'Epoch: {epoch:04d}, Trn_loss: {trn_loss:.4f}, Trn_acc: {trn_acc:.4f}, Val_acc: {val_acc:.4f}, Test_acc: {tst_acc:.4f}')


run experiments on Cora dataset
Epoch: 0000, Trn_loss: 2.1050, Trn_acc: 0.1464, Val_acc: 0.0387, Test_acc: 0.0328
Epoch: 0001, Trn_loss: 63.5351, Trn_acc: 0.2010, Val_acc: 0.1679, Test_acc: 0.1724
Epoch: 0002, Trn_loss: 96.1933, Trn_acc: 0.1522, Val_acc: 0.0498, Test_acc: 0.0640
Epoch: 0003, Trn_loss: 123.6806, Trn_acc: 0.1490, Val_acc: 0.1052, Test_acc: 0.1018
Epoch: 0004, Trn_loss: 127.6281, Trn_acc: 0.1496, Val_acc: 0.1513, Test_acc: 0.1560
Epoch: 0005, Trn_loss: 79.1536, Trn_acc: 0.2023, Val_acc: 0.1605, Test_acc: 0.1626
Epoch: 0006, Trn_loss: 82.2540, Trn_acc: 0.2132, Val_acc: 0.1236, Test_acc: 0.1166
Epoch: 0007, Trn_loss: 83.4842, Trn_acc: 0.2261, Val_acc: 0.2786, Test_acc: 0.2611
Epoch: 0008, Trn_loss: 57.1321, Trn_acc: 0.3205, Val_acc: 0.6476, Test_acc: 0.6371
Epoch: 0009, Trn_loss: 56.0608, Trn_acc: 0.3359, Val_acc: 0.6863, Test_acc: 0.6519
Epoch: 0010, Trn_loss: 53.3311, Trn_acc: 0.3911, Val_acc: 0.5812, Test_acc: 0.6026
Epoch: 0011, Trn_loss: 44.7101, Trn_acc: 0.5106, Val_a